# II.4 Dual Numbers (cont.)


Recall: dual numbers are $a + b ε$ where $ε^2 = 0$.

Similar to complex numbers: $a + b {\rm i}$ where ${\rm i}^2 = -1$.

We showed last lecture that if $p$ is polynomial that
$$
p(a + bε) = p(a) + bp'(a)ε.
$$
We want to now implement dual numbers on a computer to compute derivatives.

**Forward-mode automatic differentiation**


In [51]:
import Base: show, +, *, /, -, exp, ^ # ring operations

# user Taylor series exp(x) ≈ Σ_{k=0}^n x^k/k!
function exp_t(x, n)
    ret = 0
    summand = 1
    for k = 0:n
        ret += summand
        summand *= x/(k+1) # uses Float64 arithmetic
    end
    return ret
end

exp_t (generic function with 1 method)

In [10]:
# complex number in Julia
im^2 # this is i, i.e. im^2
z = 1 + 2im # this means 1 + 2i

1 + 2im

In [13]:
# typeof(z) == Complex{Int} since the real/imag parts are Int.
# We can create also as follows
z = Complex(1, 2) # same as 1+2im
# A complex number has two fields corresponding to the real and imag parts:
z.re, z.im

(1, 2)

In [15]:
# * is overloaded to perform complex multiplication
z * z

-3 + 4im

In [17]:

# When we add a Float64 and a Complex{Int} we get a Complex{Float64} ≡ ComplexF64
z = 1.4 + 2im
typeof(z)

ComplexF64 (alias for Complex{Float64})

In [59]:
# Dual(a,b) represents a + b*ε
struct Dual{T}
    a::T # this means a is type T, usually Float64
    b::T
end

# This is called if a and b are different types. If they are the same type `T`
# then it creates a Dual{T}. This can be made explicit via Dual{T}(a,b)
function Dual(a, b)
    T = promote_type(typeof(a), typeof(b)) # usually find bigger type, Float64
    Dual{T}(a, b) # creates Dual{T}, converts a and b to type T
end

# const means the variable cannot change
const ε = Dual(0,1)



# overload show to change the display (non-examinable)
show(io::IO, x::Dual) = print(io, string(x.a) * " + " * string(x.b) * "ε")

# When adding a Real (say an Int) to a Dual, we modify only `a`
+(x::Real, y::Dual) = Dual(x+y.a, y.b)
+(x::Dual, y::Dual) = Dual(x.a+y.a, x.b+y.b)
# This allows us to negate dual numbers
-(x::Dual) = Dual(-x.a, -x.b)

# Dividing/multiplying by a Real divides/multiplies each component as its a vector sace
/(x::Dual, y::Real) = Dual(x.a/y,x.b/y)
*(x::Real, y::Dual) = Dual(x*y.a, x*y.b)

# (a+b*ε)*(c+d*ε)== a*c + (b*c+a*d)ε
*(x::Dual, y::Dual) = Dual(x.a*y.a, x.b*y.a + x.a*y.b)

# to support x^2 (which is equivalent to the call ^(x, 2)), we need to overload ^. 
function ^(x::Dual, k::Integer) # Integer is a abstract type contain Int, UInt8, etc
    if k < 0
        error("Not implemented")
    elseif k == 1
        x
    else
        x^(k-1) * x
    end
end

# Can extend non-polynomials by using the "dual extension"
exp(x::Dual) = Dual(exp(x.a), x.b*exp(x.a))

exp (generic function with 15 methods)

In [42]:
# We now support all the operations used in exp_t:
exp_t(Dual(1.0,1.0), 50) # should be exp_t(1) + exp_t'(1)
# incredibly accurate!

2.7182818284590455 + 2.7182818284590455ε

In [43]:
# More accurate than divided differences (w/ quasi-optimal choice of h)
h = sqrt(eps())
(exp_t(1+h, 50)-exp_t(1, 50))/h

2.7182818353176117

In [48]:
# we can compose functions
exp(exp(1 + ε))

15.154262241479262 + 41.193555674716116ε

In [57]:
# a more complicated example
f = x -> exp(-exp(x)^2)
f(1+ε)

0.000617978989331094 + -0.009132562840255837ε

In [61]:
# lets diff. exp(-exp(exp(-exp(… exp(x))))) where the ... is say 100 time

x = 1.1 + ε
for k = 1:100
    x = exp((-1)^k * x)
end
x

1.3097995858041505 + 3.175573917841343e-23ε

In [62]:
# Lets check the accuracy using high-precision big flaots:
x = big(1.1) + ε
for k = 1:100
    x = exp((-1)^k * x)
end
x # amazingly, the derivative is accurate to 15 digits!

1.309799585804150477669227636073309794231627797088119910659089904813984183832131 + 3.175573917841337600563247262642274789510873995540578821780919894216315914212658e-23ε